In [ ]:
#BERT understanding
#https://jalammar.github.io/illustrated-bert/

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

texts = [
    "This movie is great and I loved it!",
    "Terrible film, very boring.",
    "Amazing storyline and acting!",
    "I didn't enjoy this at all.",
    "Fantastic experience, highly recommend!",
    "Really bad, waste of time.",

    "One of the best movies I've seen this year!",
    "Completely disappointing and predictable.",
    "Brilliant direction and stunning visuals.",
    "I fell asleep halfway through, so dull.",
    "Heartwarming and beautifully shot.",
    "Poor acting and weak script.",

    "Absolutely loved the plot twists!",
    "Not worth the hype at all.",
    "Engaging from start to finish!",
    "The worst film I’ve ever watched.",
    "Incredible performances by the cast!",
    "Script was a mess and pacing was off."
]
labels = [1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0]


# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize data
def tokenize_texts(texts, max_len=128):
    encodings = tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )
    return encodings['input_ids'], encodings['attention_mask']

train_input_ids, train_attention_mask = tokenize_texts(train_texts)
val_input_ids, val_attention_mask = tokenize_texts(val_texts)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int32)

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels
)).batch(2)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': val_input_ids, 'attention_mask': val_attention_mask},
    val_labels
)).batch(2)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=2
)

# Inference function
def predict_sentiment(text, model, tokenizer, max_len=128):
    encodings = tokenizer(
        [text],
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )
    outputs = model({'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']})
    logits = outputs.logits
    prediction = tf.argmax(logits, axis=-1).numpy()[0]
    print(logits)
    return "Positive" if prediction == 1 else "Negative"

# Test inference
test_text = "This is an awesome movie!"
result = predict_sentiment(test_text, model, tokenizer)
print(f"Text: {test_text}")
print(f"Predicted Sentiment: {result}")

In [ ]:
# Test inference
test_text = "Movie is very boring in first half and in the second half the movie is worth watching "
result = predict_sentiment(test_text, model, tokenizer)
print(f"Text: {test_text}")
print(f"Predicted Sentiment: {result}")

In [ ]:
# Inference function
def predict_sentiment(text, model, tokenizer, max_len=128):
    encodings = tokenizer(
        [text],
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )
    outputs = model({'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']})
    logits = outputs.logits
    print(outputs)

    # Calculate softmax probabilities
    probabilities = tf.nn.softmax(logits, axis=-1)
    print("Softmax Probabilities:", probabilities)

    prediction = tf.argmax(probabilities, axis=-1).numpy()[0]
    print("Predicted Class Index:", prediction)

    return "Positive" if prediction == 1 else "Negative"

# Test inference
test_text = "worst movie"
result = predict_sentiment(test_text, model, tokenizer)
print(f"Text: {test_text}")
print(f"Predicted Sentiment: {result}")

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Dataset
texts = [
    "This movie is great and I loved it!",
    "Terrible film, very boring.",
    "Amazing storyline and acting!",
    "I didn't enjoy this at all.",
    "Fantastic experience, highly recommend!",
    "Really bad, waste of time.",
    "One of the best movies I've seen this year!",
    "Completely disappointing and predictable.",
    "Brilliant direction and stunning visuals.",
    "I fell asleep halfway through, so dull.",
    "Heartwarming and beautifully shot.",
    "Poor acting and weak script.",
    "Absolutely loved the plot twists!",
    "Not worth the hype at all.",
    "Engaging from start to finish!",
    "The worst film I’ve ever watched.",
    "Incredible performances by the cast!",
    "Script was a mess and pacing was off."
]
labels = [1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0]

# Train/Val Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_texts(texts, max_len=128):
    encodings = tokenizer(
        texts,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    return encodings['input_ids'], encodings['attention_mask']

# Tokenize data
train_input_ids, train_attention_mask = tokenize_texts(train_texts)
val_input_ids, val_attention_mask = tokenize_texts(val_texts)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int32)

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels
)).batch(2).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': val_input_ids, 'attention_mask': val_attention_mask},
    val_labels
)).batch(2).prefetch(tf.data.AUTOTUNE)

# ===== Custom Model with Softmax =====

# Load base BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Input layers with dynamic shape (fix for error)
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# Get pooled output from BERT
bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
pooled_output = bert_outputs.pooler_output  # CLS token output

# Optional dropout
dropout = tf.keras.layers.Dropout(0.3)(pooled_output)

# Classification head with softmax
output = tf.keras.layers.Dense(2, activation='softmax')(dropout)

# Build model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ===== Train the Model =====
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=2
)

# ===== Inference Function =====
def predict_sentiment(text, model, tokenizer, max_len=128):
    encodings = tokenizer(
        [text],
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    probs = model({'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']}).numpy()
    prediction = np.argmax(probs, axis=-1)[0]
    print(f"Probabilities: {probs}")
    return "Positive" if prediction == 1 else "Negative"

# ===== Test Inference =====
test_text = "This is an awesome movie!"
result = predict_sentiment(test_text, model, tokenizer)
print(f"\nText: {test_text}")
print(f"Predicted Sentiment: {result}")
